<a href="https://colab.research.google.com/github/JulioHaro93/PracticasTesisKeras/blob/main/aprendizaje_por_transferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install tensorflow==2.12

Instalamos tqdm

In [13]:
!pip install tqdm

Descargamos el dataset desde el ambiente de desarrollo de google

In [14]:
!wget --no-check-certificate \https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \-O ./cats_and_dogs_filtered.zip

--2025-05-06 22:31:49--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 192.178.218.207, 142.251.179.207, 64.233.180.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|192.178.218.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   229MB/s    in 0.3s    

2025-05-06 22:31:50 (229 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



Preprocesamiento de datos

In [15]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator #OJO AQUÍ

%matplotlib inline
tf.__version__

'2.12.0'

Descomprimir dataset de perros vs Gatos

In [17]:
dataser_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataser_path, mode="r")

zip_object.extractall("./")
zip_object.close()

**Es importante automatizar el preprocesado con la gfunción de ImageDataGenerator, eso les da el tamaño y la forma homogénea**

In [18]:
dataser_path_new = "./cats_and_dogs_filtered/"

train_dir = os.path.join(dataser_path_new, "train")
validation_dir = os.path.join(dataser_path_new, "validation")